# Fine-tuning the latest Google Gemma model locally using MLX

## 1. Preparation


In [58]:
# !pip install -Uqq mlx mlx_lm transformers datasets

## 2. Using MLX to Run Inference with Gemma Model using MLX

In [1]:
from mlx_lm import generate, load

model, tokenizer = load("mlx-community/gemma-2-27b-it-4bit")
# https://huggingface.co/mlx-community/gemma-2-27b-4bit 


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
messages = [{"role": "user", "content": "What are the minimum size requirements for the marks and the UN packaging symbol on pressure receptacles, and how do these requirements differ based on the diameter of the pressure receptacle?"}]
result = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(result)

<bos><start_of_turn>user
What are the minimum size requirements for the marks and the UN packaging symbol on pressure receptacles, and how do these requirements differ based on the diameter of the pressure receptacle?<end_of_turn>
<start_of_turn>model



In [4]:
# Generating without adding a prompt template manually
prompt = """
What are the minimum size requirements for the marks and the UN packaging symbol on pressure receptacles, and how do these requirements differ based on the diameter of the pressure receptacle per IMDG code?
""".strip()
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    verbose=True,  # Set to True to see the prompt and response
    temp=0.0,
    max_tokens=1024,
)

Prompt: What are the minimum size requirements for the marks and the UN packaging symbol on pressure receptacles, and how do these requirements differ based on the diameter of the pressure receptacle per IMDG code?


The IMDG Code (International Maritime Dangerous Goods Code) outlines the marking and labeling requirements for pressure receptacles.

**Minimum Size Requirements for Marks and UN Packaging Symbol:**

The minimum size requirements for marks and the UN packaging symbol on pressure receptacles are dependent on the diameter of the receptacle.

* **Receptacles with a diameter of less than 140 mm:**

    * The UN packaging symbol must have a minimum dimension of 6 mm.
    * Other marks (e.g., manufacturer's name, serial number, test pressure, etc.) must be clearly visible and legible.

* **Receptacles with a diameter of 140 mm or more:**

    * The UN packaging symbol must have a minimum dimension of 12 mm.
    * Other marks must be clearly visible and legible, with a minimum he

## 3. Generating training dataset from PDF file

In [6]:
import PyPDF2
import csv, re
import random
from tqdm import tqdm
import time
from transformers import AutoTokenizer
from mlx_lm import load, generate
import subprocess

def run_command_with_live_output(command: list[str]) -> None:
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    err_output = process.stderr.read()
    if err_output:
        print(err_output)

def extract_text_from_pdf(pdf_path):
    print("Extracting text from PDF...")
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in tqdm(reader.pages, desc="Processing pages"):
            text += page.extract_text()
    print("Text extraction completed")
    return text

def create_chunks(text, min_chunk_size=200, max_chunk_size=1000):
    print("Splitting text into chunks...")
    chunks = []
    current_chunk = ""
    sentences = text.split('.')
    for sentence in tqdm(sentences, desc="Creating chunks"):
        if len(current_chunk) + len(sentence) > max_chunk_size and len(current_chunk) >= min_chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence + '.'
    if current_chunk:
        chunks.append(current_chunk.strip())
    print(f"Number of chunks created: {len(chunks)}")
    return chunks

def load_model_and_tokenizer(model_path):
    print("Loading model and tokenizer...")
    model, tokenizer = load(model_path)
    return model, tokenizer

def generate_question_response_pair(chunk, model, tokenizer, max_tokens=512):
    instruction = """
    Based on the IMDG code, create a practical question and provide a detailed, informative answer. 
    All the questions should be based on the IMDG code.
    Do not include any special markers like '**' or '<end_of_turn>'
    Format your response as follows:
    Question: [Your generated question]
    Answer: [Your generated answer]
"""

    prompt = f'''<s>[INST] {instruction}\n\nText: {chunk} [/INST]\n'''
    
    # generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=max_tokens, verbose=False)

    generated_text = generate(
        model, 
        tokenizer, 
        prompt=prompt, 
        max_tokens=max_tokens, 
        temp=0.8,  # Value between 0.0 and 1.0, higher values produce more creative output
        # top_k=50,         # Number of top tokens to consider
        top_p=0.95,       # Select tokens until cumulative probability exceeds this value
        verbose=False
    )

    generated_text = generated_text.replace(prompt, "").strip()
    generated_text = re.sub(r'\*\*|<end_of_turn>', '', generated_text)
    generated_text = re.sub(r'```', '', generated_text)
    
    question_start = generated_text.find("Question:")
    answer_start = generated_text.find("Answer:")
    
    if question_start != -1 and answer_start != -1:
        question = generated_text[question_start+9:answer_start].strip()
        answer = generated_text[answer_start+7:].strip()
    else:
        question = "Unable to generate a question."
        answer = "Unable to generate a response."
    
    return question, answer

def create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000):
    start_time = time.time()
    print(f"Starting fine-tuning dataset creation (Target samples: {num_samples})")

    model_path = "mlx-community/gemma-2-27b-it-4bit"
    model, tokenizer = load_model_and_tokenizer(model_path)
    print("Model and tokenizer loaded")

    text = extract_text_from_pdf(pdf_path)
    chunks = create_chunks(text)
    
    print("Generating question-response pairs and writing to CSV...")
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['question', 'response'])
        
        for _ in tqdm(range(num_samples), desc="Generating samples"):
            chunk = random.choice(chunks)
            question, response = generate_question_response_pair(chunk, model, tokenizer)
            if question != "Unable to generate a question." and response != "Unable to generate a response.":
                writer.writerow([question, response])
            else:
                print(f"Skipping invalid sample: Q: {question}, A: {response}")
            if (_ + 1) % 100 == 0:
                print(f"Generated {_ + 1} samples...")
    
    end_time = time.time()
    print(f"Number of dataset samples generated: {num_samples}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")

# Usage example
pdf_path = "./data/IMDG.pdf"
output_file = "./data/IMDG.csv"
create_fine_tuning_dataset(pdf_path, output_file, num_samples=10000)

Starting fine-tuning dataset creation (Target samples: 10000)
Loading model and tokenizer...


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Model and tokenizer loaded
Extracting text from PDF...


Processing pages: 100%|██████████| 1098/1098 [01:05<00:00, 16.85it/s]


Text extraction completed
Splitting text into chunks...


...

Generating samples:  39%|███▉      | 3900/10000 [20:28:01<33:43:30, 19.90s/it]

Generated 3900 samples...


Generating samples:  39%|███▉      | 3902/10000 [20:28:46<36:11:10, 21.36s/it]

Skipping invalid sample: Q: Unable to generate a question., A: Unable to generate a response.


Generating samples:  39%|███▉      | 3920/10000 [20:35:16<34:13:32, 20.27s/it]

## 4. Building dataframe

In [1]:
import csv
import random
import json
import pandas as pd

# load the dataset
dataset = pd.read_csv("./data/IMDG.csv")

system_prompt = """
    You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answer or if the information is not in the provided context, clearly state "I don't have enough information to answer this question accurately. Please refer to the official IMDG Code or consult with an IMDG expert for the most up-to-date and accurate information."
    5. End each response with '–IMDGGenie'
"""

dataset["system_prompt"] = system_prompt
dataset["system_prompt"] = dataset["system_prompt"].str.strip()
dataset = dataset[["system_prompt", "question", "response"]]
dataset

,system_prompt,question,response
0,You are an IMDG Code specialist with a deep un...,"According to the IMDG code, how should a subst...",Category A toxins are assigned to UN number 34...
1,You are an IMDG Code specialist with a deep un...,What are the specific packaging requirements f...,Paragraph P600 of the IMDG Code outlines speci...
2,You are an IMDG Code specialist with a deep un...,A cargo transport unit contains both dangerous...,"Yes, you would need to placard the cargo trans..."
3,You are an IMDG Code specialist with a deep un...,Based on the IMDG Code information provided fo...,"According to the IMDG Code excerpt, Nickel Car..."
4,You are an IMDG Code specialist with a deep un...,"A shipment of silvery-white, ductile, soft met...",This material aligns with the descriptions pro...
...,...,...,...
3988,You are an IMDG Code specialist with a deep un...,"According to the IMDG Code, what are the regul...",The IMDG Code stipulates that placards indicat...
3989,You are an IMDG Code specialist with a deep un...,"A chemical shipment labeled ""SG49"" is declared...","Given the hazardous nature of the ""SG49"" corro..."
3990,You are an IMDG Code specialist with a deep un...,A mixture contains two toxic components: Subst...,"To determine the packing group, we need to cal..."
3991,You are an IMDG Code specialist with a deep un...,A package design incorporates a single waterti...,"No, we cannot assume that water will not leak ..."


In [2]:
# Count records for Unable to generate a question & Unable to generate a response
filtered_df = dataset[
    (dataset["question"] == "Unable to generate a question.") & 
    (dataset["response"] == "Unable to generate a response.")
]

count = filtered_df.shape[0]
print(f"Number of rows with the specified question and response: {count}")

Number of rows with the specified question and response: 0


In [3]:
# Count records for duplicated question and response
filtered_df = dataset[
    dataset.duplicated(subset=["question", "response"], keep=False)
]

count = filtered_df.shape[0]
print(f"Number of rows with the same question and response: {count}")

Number of rows with the same question and response: 0


In [4]:
# Drop duplicates
df = dataset.drop_duplicates(subset=["question", "response"])
df

,system_prompt,question,response
0,You are an IMDG Code specialist with a deep un...,"According to the IMDG code, how should a subst...",Category A toxins are assigned to UN number 34...
1,You are an IMDG Code specialist with a deep un...,What are the specific packaging requirements f...,Paragraph P600 of the IMDG Code outlines speci...
2,You are an IMDG Code specialist with a deep un...,A cargo transport unit contains both dangerous...,"Yes, you would need to placard the cargo trans..."
3,You are an IMDG Code specialist with a deep un...,Based on the IMDG Code information provided fo...,"According to the IMDG Code excerpt, Nickel Car..."
4,You are an IMDG Code specialist with a deep un...,"A shipment of silvery-white, ductile, soft met...",This material aligns with the descriptions pro...
...,...,...,...
3988,You are an IMDG Code specialist with a deep un...,"According to the IMDG Code, what are the regul...",The IMDG Code stipulates that placards indicat...
3989,You are an IMDG Code specialist with a deep un...,"A chemical shipment labeled ""SG49"" is declared...","Given the hazardous nature of the ""SG49"" corro..."
3990,You are an IMDG Code specialist with a deep un...,A mixture contains two toxic components: Subst...,"To determine the packing group, we need to cal..."
3991,You are an IMDG Code specialist with a deep un...,A package design incorporates a single waterti...,"No, we cannot assume that water will not leak ..."


In [5]:
# Transform Gemma prompt template (https://ai.google.dev/gemma/docs/formatting)
# {"text": "<bos><start_of_turn>user\nWhat is the capital of France?<end_of_turn>\n<start_of_turn>model\nParis is the capital of France.<end_of_turn><eos>"}

def generate_prompt(row: pd.Series) -> str:
    "Format to Gemma's chat template"
    return """<bos><start_of_turn>user
## Instructions
{}
## User
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn>""".format(row["system_prompt"], row["question"], row["response"])


df["text"] = df.apply(generate_prompt, axis=1)
print(df["text"].iloc[100])

<bos><start_of_turn>user
## Instructions
You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answer or if

In [6]:
# Split dataset to train and valid 

from pathlib import Path

Path("data").mkdir(exist_ok=True)

split_ix = int(len(df) * 0.9)
# shuffle data
data = df.sample(frac=1, random_state=42)
train, valid = data[:split_ix], data[split_ix:]

# Save train and valid dataset as jsonl files
train[["text"]].to_json("data/train.jsonl", orient="records", lines=True, force_ascii=False)
valid[["text"]].to_json("data/valid.jsonl", orient="records", lines=True, force_ascii=False)

!head -n 1 data/train.jsonl

{"text":"<bos><start_of_turn>user\n## Instructions\nYou are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.\n\n    Guidelines for answering:\n    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.\n    2. For questions about specific UN Numbers:\n       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.\n       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.\n    3. Respond in the language of the user's question. If unable to determine the language, default to English.\n    4. If you don't kn

## 5. LoRA fine-tuning

In [7]:
!python -m mlx_lm.lora --help

usage: lora.py [-h] [--model MODEL] [--train] [--data DATA]
               [--lora-layers LORA_LAYERS] [--batch-size BATCH_SIZE]
               [--iters ITERS] [--val-batches VAL_BATCHES]
               [--learning-rate LEARNING_RATE]
               [--steps-per-report STEPS_PER_REPORT]
               [--steps-per-eval STEPS_PER_EVAL]
               [--resume-adapter-file RESUME_ADAPTER_FILE]
               [--adapter-path ADAPTER_PATH] [--save-every SAVE_EVERY]
               [--test] [--test-batches TEST_BATCHES]
               [--max-seq-length MAX_SEQ_LENGTH] [-c CONFIG]
               [--grad-checkpoint] [--seed SEED] [--use-dora]

LoRA or QLoRA finetuning.

options:
  -h, --help            show this help message and exit
  --model MODEL         The path to the local model directory or Hugging Face
                        repo.
  --train               Do training
  --data DATA           Directory with {train, valid, test}.jsonl files
  --lora-layers LORA_LAYERS
                   

In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [9]:
!python -m mlx_lm.lora \
    --model mlx-community/gemma-2-27b-it-4bit \
    --train \
    --data data \
    --iters 500 \
    --batch-size 4 \
    --learning-rate 1e-5 \
    --steps-per-report 10 \
    --steps-per-eval 10 \
    --adapter-path checkpoints/adapters \
    # --resume-adapter-file checkpoints/600_adapters.npz \
    --save-every 10 \
    --max-seq-length 2048 \
    --seed 42 \
    --lora-layers 16

Loading pretrained model
Fetching 9 files: 100%|████████████████████████| 9/9 [00:00<00:00, 77672.30it/s]
Loading datasets
Training
Trainable parameters: 0.007% (1.966M/27227.128M)
Starting training..., iters: 500
Iter 1: Val loss 2.061, Val took 400.467s
Iter 10: Val loss 1.522, Val took 398.144s
Iter 10: Train loss 1.815, Learning Rate 1.000e-05, It/sec 0.486, Tokens/sec 885.188, Trained Tokens 18218, Peak mem 58.444 GB
Iter 20: Val loss 1.172, Val took 415.262s
Iter 20: Train loss 1.326, Learning Rate 1.000e-05, It/sec 0.232, Tokens/sec 491.767, Trained Tokens 39452, Peak mem 60.797 GB
Iter 30: Val loss 0.811, Val took 407.030s
Iter 30: Train loss 0.924, Learning Rate 1.000e-05, It/sec 0.429, Tokens/sec 821.017, Trained Tokens 58606, Peak mem 60.797 GB
Iter 40: Val loss 0.578, Val took 402.604s
Iter 40: Train loss 0.682, Learning Rate 1.000e-05, It/sec 0.460, Tokens/sec 918.351, Trained Tokens 78561, Peak mem 60.797 GB
Iter 50: Val loss 0.535, Val took 396.426s
Iter 50: Train loss 0

In [ ]:
# !python -m mlx_lm.lora \
#     --model mlx-community/gemma-2-27b-it-4bit \
#     --train \
#     --data data \
#     --iters 600 \
#     --batch-size 4 \
#     --learning-rate 1e-5 \
#     --steps-per-report 10 \
#     --steps-per-eval 10 \
#     --adapter-path checkpoints/adapters \
#     --resume-adapter-file checkpoints/adapters/0000100_adapters.safetensors \
#     --save-every 10 \
#     --max-seq-length 2048 \
#     --seed 42 \
#     --lora-layers 16

## 6. Inference with fine-tuned model

In [18]:
# System prompt

system_prompt = df["system_prompt"].unique()[-1]
print(system_prompt)

You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answer or if the information is not in the provided c

In [30]:
question = "What are the minimum wall thickness requirements for IBCs used for transporting liquids according to the IMDG Code?"


def format_prompt(system_prompt: str, question: str) -> str:
    "Format the question to the format of the dataset we fine-tuned to."
    return """<bos><start_of_turn>user
## Instructions
{}
## User
{}<end_of_turn>
<start_of_turn>model
""".format(
        system_prompt, question
    )


print(format_prompt(system_prompt, question))

<bos><start_of_turn>user
## Instructions
You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answer or if

In [31]:
# Load the fine-tuned model with LoRA weights
model_lora, tokenizer = load(
    "mlx-community/gemma-2-27b-it-4bit",
    adapter_path="./checkpoints/adapters",
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [32]:
response = generate(
    model_lora,
    tokenizer,
    prompt=format_prompt(system_prompt, question),
    verbose=True,
    temp=0.5,
    max_tokens=1024,
)

Prompt: <bos><start_of_turn>user
## Instructions
You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answ

In [33]:
response = generate(
    model,
    tokenizer,
    prompt=format_prompt(system_prompt, question),
    verbose=True,
    temp=0.5,
    max_tokens=1024,
)

Prompt: <bos><start_of_turn>user
## Instructions
You are an IMDG Code specialist with a deep understanding of the International Maritime Dangerous Goods (IMDG) Code. Your task is to provide detailed and accurate information in response to user questions about IMDG code.

    Guidelines for answering:
    1. Present information in a clear, concise, and easily understandable manner, using bullet points for organization.
    2. For questions about specific UN Numbers:
       - Provide details on: UN No., Proper Shipping Name, Class, Subsidiary hazard, Packing Group, Special Provisions, Limited Quantity, Excepted Quantity, Packing Instructions, Stowage and handling, Segregation, and Properties and observations.
       - If this information is not in the provided context, state that you need to refer to the official IMDG Code for accurate details.
    3. Respond in the language of the user's question. If unable to determine the language, default to English.
    4. If you don't know the answ

In [18]:
!python -m mlx_lm.generate \
    --model mlx-community/gemma-2-27b-it-4bit \
    --adapter-path checkpoints/adapters \
    --prompt "What are the normal working hours for customs at Dar es Salaam Port?" \
    --max-tokens 256 \
    --temp 0.5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 9 files: 100%|████████████████████████| 9/9 [00:00<00:00, 26696.42it/s]
Prompt: <bos><start_of_turn>user
What are the normal working hours for customs at Dar es Salaam Port?<end_of_turn>
<start_of_turn>model

I cannot provide specific real-time information like working hours for government offices. Working hours are subject to change and are best obtained directly from the source.<end_of_turn>
Prompt: 24 tokens, 12.770 tokens-per-sec
Generation: 32 tokens, 13.584 tokens-per-sec
Peak memory: 14.642 GB


In [12]:
!python -m mlx_lm.generate --help

usage: generate.py [-h] [--model MODEL] [--adapter-path ADAPTER_PATH]
                   [--trust-remote-code] [--eos-token EOS_TOKEN]
                   [--prompt PROMPT] [--max-tokens MAX_TOKENS] [--temp TEMP]
                   [--top-p TOP_P] [--seed SEED] [--ignore-chat-template]
                   [--use-default-chat-template] [--colorize]
                   [--cache-limit-gb CACHE_LIMIT_GB]

LLM inference script

options:
  -h, --help            show this help message and exit
  --model MODEL         The path to the local model directory or Hugging Face
                        repo.
  --adapter-path ADAPTER_PATH
                        Optional path for the trained adapter weights and
                        config.
  --trust-remote-code   Enable trusting remote code for tokenizer
  --eos-token EOS_TOKEN
                        End of sequence token for tokenizer
  --prompt PROMPT       Message to be processed by the model
  --max-tokens MAX_TOKENS, -m MAX_TOKENS
               